1: Install and Import Required Modules and Libraries

In [2]:
%pip install pandas numpy nltk scikit-learn
%pip install -U bnlp_toolkit
#%pip install banglanlp

import pandas as pd
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from bnlp import BengaliCorpus as corpus # Import the BengaliCorpus class
from bnlp import NLTKTokenizer
#from banglanlp.tokenizer import Tokenizer

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\thetr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thetr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\thetr\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2: Load the newspaper dataset

In [3]:
# Load the CSV file
main_data = pd.read_csv("data.csv", encoding='utf-8')

# Display basic info and a sample of the data
main_data.info()
main_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30001 entries, 0 to 30000
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   content         29994 non-null  object
 1   category        30000 non-null  object
 2   author          30000 non-null  object
 3   published_date  30000 non-null  object
 4   title           30000 non-null  object
 5   url             30000 non-null  object
dtypes: object(6)
memory usage: 1.4+ MB


,content,category,author,published_date,title,url
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,bangladesh,গাজীপুর প্রতিনিধি,"০৪ জুলাই ২০১৩, ২৩:২৬","কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক...",http://www.prothom-alo.com/bangladesh/article/...
1,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,sports,অনলাইন ডেস্ক,"০৪ জুলাই ২০১৩, ২৩:০৯",সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,http://www.prothom-alo.com/sports/article/19028
2,জাতীয় সংসদে বিএনপি চেয়ারপারসন ও বিরোধীদলীয় ...,bangladesh,নিজস্ব প্রতিবেদক,"০৪ জুলাই ২০১৩, ২২:২৫",সংসদে খালেদার অভিযোগের জবাব দিয়েছে ভারত,http://www.prothom-alo.com/bangladesh/article/...
3,সহজ পাসওয়ার্ডের কারণে অনলাইন অ্যাকাউন্ট সহজেই...,technology,অনলাইন ডেস্ক,"০৪ জুলাই ২০১৩, ২১:৩৭",পাসওয়ার্ড ভুলে যান!,http://www.prothom-alo.com/technology/article/...
4,কম্পিউটার মাউসের উদ্ভাবক ডগলাস অ্যাঙ্গেলবার্ট ...,technology,অনলাইন ডেস্ক,"০৪ জুলাই ২০১৩, ২১:৩৫",চলে গেলেন মাউস উদ্ভাবক,http://www.prothom-alo.com/technology/article/...
...,...,...,...,...,...,...
29996,সদ্য অনুষ্ঠিত সাধারণ নির্বাচনে ব্যাপক ভোট জালি...,international,-1,"২২ নভেম্বর ২০১৩, ০০:২৫",জালিয়াতির অভিযোগে নির্বাচন প্রত্যাখ্যান প্রচণ্ডের,http://www.prothom-alo.com/international/artic...
29997,কবি সুফিয়া কামাল রাষ্ট্রপতি জিয়াউর রহমানের সাম...,bangladesh,নিজস্ব প্রতিবেদক,"২২ নভেম্বর ২০১৩, ০০:২৪",সুফিয়া কামালের মতো স্পষ্ট অবস্থান নেওয়ার আহ্বান,http://www.prothom-alo.com/bangladesh/article/...
29998,টিকফা চুক্তি দেশের সার্বভৌমত্বের ওপর একটি বড় আ...,bangladesh,-1,"২২ নভেম্বর ২০১৩, ০০:২২",টিকফা চুক্তি বাতিল করার আহ্বান,http://www.prothom-alo.com/bangladesh/article/...
29999,বিএনপির নেতা সালাউদ্দিন কাদের (সাকা) চৌধুরীর ব...,bangladesh,আদালত প্রতিবেদক,"২২ নভেম্বর ২০১৩, ০০:২১",সাকার আইনজীবী ফখরুলের রিমান্ড শুনানি রোববার,http://www.prothom-alo.com/bangladesh/article/...


3. Articles to sentence for grammar and spell checking

In [4]:
def bangla_articles_sentence(text):
    # Regular expression pattern for Bangla sentence boundaries (assuming sentences end with punctuation)
    sentences = re.split(r'(?<=[।?!])', text)
    return [sentence.strip() for sentence in sentences if sentence]

# Create a new DataFrame with one sentence per row
df = pd.DataFrame({
    'sentence': [sentence for article in main_data['content'].fillna('') for sentence in bangla_articles_sentence(article)]
})

df.info()
df
#print(df['sentence'].iloc[5])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603624 entries, 0 to 603623
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sentence  603624 non-null  object
dtypes: object(1)
memory usage: 4.6+ MB


,sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।
...,...
603619,রাষ্ট্রীয় বার্তা সংস্থা সিনহুয়া গতকাল বৃহস্পতি...
603620,একপর্যায়ে চিকিৎসকেরা তার মৃত্যু হয়েছে বলে ঘোষণ...
603621,প্রাদেশিক রাজধানী হেফেইতে অন্ত্যেষ্টিক্রিয়ার স...
603622,তবে ঠিক কত সময় শিশুটিকে অন্ত্যেষ্টিক্রিয়ার কক্...


4: Text Cleaning and Normalization

In [5]:
# Count duplicate rows in the 'sentence' column
duplicate_count = df['sentence'].duplicated().sum()
print("Number of duplicate sentences:", duplicate_count)

# # Display duplicate rows in the 'sentence' column
# duplicate_sentences = df[df['sentence'].duplicated(keep=False)] \
#     .groupby('sentence').size().reset_index(name='count') \
#     .sort_values(by='count', ascending=False)
# # Display sentences with more than one occurrence
# print("Duplicate sentences and their counts:")
# print(duplicate_sentences[duplicate_sentences['count'] > 1])

Number of duplicate sentences: 38187


In [6]:
#remove duplicate values
df.drop_duplicates(inplace=True)

print('Number of available rows after dropping duplicates:',df.shape[0])

Number of available rows after dropping duplicates: 565437


In [7]:
# Count rows with only one word, one letter, only punctuation, or only whitespace
counts = {
    "one_word": df['sentence'].str.match(r'^[\u0980-\u09FF]+$|^\w+$').sum(),
    "one_letter": df['sentence'].str.match(r'^[\u0980-\u09FF]$|^[^\W\d_]$').sum(),
    "only_punctuation": df['sentence'].str.match(r'^[^\w\s]+$').sum(),
    "only_space": df['sentence'].str.match(r'^\s*$').sum(),
}
print("Counts:", counts)

# Drop rows in 'sentence' column that contain only one word, one letter, punctuation, or whitespace
df = df[~df['sentence'].str.match(r'^[\u0980-\u09FF]+$|^\w+$|^[\u0980-\u09FF]$|^[^\W\d_]$|^[^\w\s]+$|^\s*$', na=False)]

df

Counts: {'one_word': 25, 'one_letter': 5, 'only_punctuation': 14, 'only_space': 0}


,sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।
...,...
603618,চীনের আনুহি প্রদেশে ঘটেছে এ ঘটনা।
603619,রাষ্ট্রীয় বার্তা সংস্থা সিনহুয়া গতকাল বৃহস্পতি...
603620,একপর্যায়ে চিকিৎসকেরা তার মৃত্যু হয়েছে বলে ঘোষণ...
603621,প্রাদেশিক রাজধানী হেফেইতে অন্ত্যেষ্টিক্রিয়ার স...


In [18]:
# Keep only Bangla characters and remove punctuation or symbols or unnecessary whitespace
def clean_text(text):
    bangla_punctuation = r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~।ঃ' #corpus.punctuations
    text = re.sub(r'[^\u0980-\u09FF\s' + bangla_punctuation + ']', '', text)  # Keep only Bangla characters without punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

df['cleaned_text'] = df['sentence'].apply(clean_text) 
df[['sentence', 'cleaned_text']].info()
df[['sentence', 'cleaned_text']].head()

<class 'pandas.core.frame.DataFrame'>
Index: 565398 entries, 0 to 603622
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   sentence      565398 non-null  object
 1   cleaned_text  565398 non-null  object
dtypes: object(2)
memory usage: 12.9+ MB


,sentence,cleaned_text
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,এতে আতঙ্কিত হওয়ার কিছু নেই।
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।


5: Tokenization

In [9]:
# Initialize tokenizer
tokenizer = NLTKTokenizer()

def tokenize_text(text):
    tokens = tokenizer.word_tokenize(text)
    return tokens

df['tokens'] = df['cleaned_text'].apply(tokenize_text)
df[['cleaned_text', 'tokens']].head()

,cleaned_text,tokens
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[গাজীপুরের, কালিয়াকৈর, উপজেলার, তেলিরচালা, এল..."
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[এ, ঘটনায়, বিক্ষোভ, করেছেন, ওই, কারখানার, শ্র..."
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[সফিপুর, মডার্ন, হাসপাতালের, জরুরি, বিভাগের, চ..."
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[এতে, আতঙ্কিত, হওয়ার, কিছু, নেই, ।]"
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,"[অসুস্থদের, চিকিত্সা, দেওয়া, হয়েছে, ।]"


6. Add Bangla POS Tagger

In [10]:
from bnlp import BengaliPOS

# Instantiate the BengaliPOS tagger
bn_pos = BengaliPOS()

# Apply POS tagging: join tokens into a sentence before passing to bn_pos.tag()
df['pos_tags'] = df['cleaned_text'].apply(lambda tokens: bn_pos.tag(tokens))

# Display the DataFrame with POS tags
df[['cleaned_text', 'pos_tags']].head()

,cleaned_text,pos_tags
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[(গাজীপুরের, NC), (কালিয়াকৈর, NC), (উপজেলার, ..."
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[(এ, DAB), (ঘটনায়, NC), (বিক্ষোভ, NC), (করেছে..."
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[(সফিপুর, NP), (মডার্ন, NC), (হাসপাতালের, NC),..."
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[(এতে, PPR), (আতঙ্কিত, JJ), (হওয়ার, NC), (কিছ..."
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,"[(অসুস্থদের, NC), (চিকিত্সা, NC), (দেওয়া, NV)..."


7. Generate incorrect sentence for grammar checking

In [11]:
import random

# Function to introduce a simple grammatical error by shuffling words
def introduce_errors(sentence):
    # Split sentence into words, ignoring punctuation
    words = re.findall(r'\b[\u0980-\u09FF]+\b', sentence)
    if len(words) > 1:
        # Shuffle words or remove one randomly to simulate error
        if random.choice([True, False]):
            random.shuffle(words)
        else:
            words.pop(random.randint(0, len(words) - 1))
    return ' '.join(words)

# Apply the function to generate incorrect sentences
df['incorrect_sentence'] = df['cleaned_text'].apply(introduce_errors)
df[['cleaned_text', 'incorrect_sentence']]

,cleaned_text,incorrect_sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,উপজেলার একট কালিয়াকৈর অসুস্থ খেয ৫০০ রাতের কা...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,এ ঘটনায বিক্ষোভ করেছেন কারখানার শ্রমিকের
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,প্রথম পড়েছেন বলেন আল শ্রমিকের ডটকমক জরুর অসুস...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,কিছ এত হওয়ার আতঙ্কিত নেই
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,অসুস্থদের চিকিত্স দেওয
...,...,...
603618,চীনের আনুহি প্রদেশে ঘটেছে এ ঘটনা।,চীনের প্রদেশ ঘটেছ এ ঘটন
603619,রাষ্ট্রীয় বার্তা সংস্থা সিনহুয়া গতকাল বৃহস্পতি...,রাষ্ট্রীয় বার্ত সংস্থ সিনহুয় গতকাল জানিয়েছ আনু...
603620,একপর্যায়ে চিকিৎসকেরা তার মৃত্যু হয়েছে বলে ঘোষণ...,একপর্যায় চিকিৎসকের মৃত্য হয়েছ বল ঘোষণ করেন
603621,প্রাদেশিক রাজধানী হেফেইতে অন্ত্যেষ্টিক্রিয়ার স...,সঙ্গ আওয়াজ প্রাদেশিক শিশুটির বুধবার গত যুক্ত অ...


In [12]:
print(df['sentence'].iloc[5])
print(df['incorrect_sentence'].iloc[5])

কারখানার শ্রমিক ও পুলিশ সূত্রে জানা যায়, উপজেলার তেলিরচালা এলাকার সেজাদ সোয়েটার লিমিটেড কারখানার শ্রমিকদের আজ রাত সাড়ে সাতটার দিকে টিফিন দেওয়া হয়।
কারখানার শ্রমিক ও পুলিশ সূত্র জান যায উপজেলার তেলিরচাল এলাকার সেজাদ সোয়েটার লিমিটেড কারখানার আজ রাত সাড সাতটার দিক টিফিন দেওয হয


8. Generate Synthetic Errors for the spell correction phase

In [13]:
import random

# Define a Function to Generate Synthetic Errors
def introduce_spelling_errors(word):
    """Randomly introduce an error into the given word."""
    if len(word) < 2:
        return word  # skip very short words
    
    error_type = random.choice(['insert', 'delete', 'substitute', 'transpose'])
    
    if error_type == 'insert':
        # Randomly insert a character
        pos = random.randint(0, len(word))
        char = chr(random.randint(2432, 2559))  # Bangla Unicode range
        new_word = word[:pos] + char + word[pos:]
        
    elif error_type == 'delete':
        # Randomly delete a character
        pos = random.randint(0, len(word) - 1)
        new_word = word[:pos] + word[pos+1:]
        
    elif error_type == 'substitute':
        # Substitute a character
        pos = random.randint(0, len(word) - 1)
        char = chr(random.randint(2432, 2559))
        new_word = word[:pos] + char + word[pos+1:]
        
    elif error_type == 'transpose':
        # Transpose two consecutive characters
        if len(word) > 1:
            pos = random.randint(0, len(word) - 2)
            new_word = word[:pos] + word[pos+1] + word[pos] + word[pos+2:]
        else:
            new_word = word
            
    return new_word

# Apply the Function to Your DataFrame
def add_spelling_errors_to_sentence(tokens):
    return [introduce_spelling_errors(word) for word in tokens]

# Apply the error introduction function to each sentence
df['synth_spelling_errors'] = df['tokens'].apply(add_spelling_errors_to_sentence)

# Convert tokens with errors back to a sentence
df['synth_spelling_errors'] = df['synth_spelling_errors'].apply(lambda tokens: ' '.join(tokens))

df[['sentence', 'synth_spelling_errors']].head()

,sentence,synth_spelling_errors
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,গা৳ীপুরের কালিয়াৈকর পউজেলার তলিরচালা োএলাকায়...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,এ ঘখটনায় বিক৹ষোভ রেছেন ইও কার৾ানার ্শরমিকেরা ।
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,সৃিপুর মডজর্ন হাসপ৛তালের জুররি বি঻ভাগের চিহিত্...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,এতঽ আতঙ্িকত হঝয়ার কছিু নে ।
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,অসুস্থদর চিকতি্সা দেও়া হয়েছওে ।


9. Double Metaphone-inspired phonetic encoding function for Bangla

In [19]:
"""Generate a phonetic encoding for Bangla words based on specified rules."""
def bangla_phonetic_encoding(word):
    phonetic_map = {
        # Special Characters and Signs
        '্': '',   # Virama/Hasant (code 0)
        'ঁ': '',   # Candrabindu (code 0)
        'ং': 'N',  # Anusvara
        'ঃ': 'H',  # Visarga

        # Vowels and Signs
        'অ': 'A', 'আ': 'AA', 'া': 'AA', 
        'ই': 'I', 'ি': 'I', 'ঈ': 'II', 'ী': 'II',
        'উ': 'U', 'ু': 'U', 'ঊ': 'UU', 'ূ': 'UU',
        'ঋ': 'RI', 'র': 'R', 'ৃ': 'RI',
        'এ': 'E', 'ে': 'E', 'ঐ': 'OI', 'ৈ': 'OI',
        'ও': 'O', 'ো': '', 'ঔ': 'OU', 'ৌ': 'OU',

        # Consonants
        'ক': 'K', 'খ': 'KH', 'গ': 'G', 'ঘ': 'GH', 'ঙ': 'NG',
        'চ': 'C', 'ছ': 'CH', 'জ': 'J', 'ঝ': 'JH', 'য': 'Y',
        'ট': 'T', 'ঠ': 'TH', 'ড': 'D', 'ঢ': 'DH', 'ণ': 'N',
        'ত': 'T', 'থ': 'TH', 'দ': 'D', 'ধ': 'DH', 'ন': 'N',
        'প': 'P', 'ফ': 'PH', 'ব': 'B', 'ভ': 'BH', 'ম': 'M',
        'শ': 'SH', 'ষ': 'SH', 'স': 'S', 'হ': 'H',

        # Additional Characters
        'ড়': 'RH', 'ঢ়': 'RH', 'য়': 'Y', 'ঞ': 'NY',
        'ল': 'L', 'ৠ': 'RR', '০': '0', '১': '1', 
        '২': '2', '৩': '3', '৪': '4', '৫': '5', '৬': '6', 
        '৭': '7', '৮': '8', '৯': '9'
    }
    
    encoded_word = []
    
    for char in word:
        if char in phonetic_map:
            encoded_word.append(phonetic_map[char])
    
    # Join encoded characters to form the phonetic encoding
    return ''.join(encoded_word)

# Apply phonetic encoding to each word in the tokens column
df['phonetic_tokens'] = df['tokens'].apply(
    lambda tokens: [bangla_phonetic_encoding(word) for word in tokens]
)

# Convert the phonetic tokens back to a sentence for easier reading
df['phonetic_encoded_sentence'] = df['phonetic_tokens'].apply(
    lambda tokens: ' '.join(tokens)
)

# Display the first few rows
df[['sentence', 'phonetic_tokens', 'phonetic_encoded_sentence']].head()

,sentence,phonetic_tokens,phonetic_encoded_sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[GAAJIIPURER, KAALIYAAKOIR, UPJELAAR, TELIRCAA...",GAAJIIPURER KAALIYAAKOIR UPJELAAR TELIRCAALAA ...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[E, GHTNAAY, BIKSHBH, KRECHEN, OI, KAARKHAANAA...",E GHTNAAY BIKSHBH KRECHEN OI KAARKHAANAAR SHRM...
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[SPHIPUR, MDAARN, HAASPAATAALER, JRURI, BIBHAA...",SPHIPUR MDAARN HAASPAATAALER JRURI BIBHAAGER C...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[ETE, AATNGKIT, HOYAAR, KICHU, NEI, ]",ETE AATNGKIT HOYAAR KICHU NEI
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,"[ASUSTHDER, CIKITSAA, DEOYAA, HYECHE, ]",ASUSTHDER CIKITSAA DEOYAA HYECHE


10: Save Preprocessed Sentence Data

In [20]:
df[['sentence', 'cleaned_text', 'tokens']].to_csv("sentence_data.csv", index=False)
print("Preprocessed sentence data saved to 'sentence_data.csv'")

df

Preprocessed sentence data saved to 'sentence_data.csv'


,sentence,cleaned_text,tokens,pos_tags,incorrect_sentence,synth_spelling_errors,phonetic_tokens,phonetic_encoded_sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[গাজীপুরের, কালিয়াকৈর, উপজেলার, তেলিরচালা, এল...","[(গাজীপুরের, NC), (কালিয়াকৈর, NC), (উপজেলার, ...",উপজেলার একট কালিয়াকৈর অসুস্থ খেয ৫০০ রাতের কা...,গা৳ীপুরের কালিয়াৈকর পউজেলার তলিরচালা োএলাকায়...,"[GAAJIIPURER, KAALIYAAKOIR, UPJELAAR, TELIRCAA...",GAAJIIPURER KAALIYAAKOIR UPJELAAR TELIRCAALAA ...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[এ, ঘটনায়, বিক্ষোভ, করেছেন, ওই, কারখানার, শ্র...","[(এ, DAB), (ঘটনায়, NC), (বিক্ষোভ, NC), (করেছে...",এ ঘটনায বিক্ষোভ করেছেন কারখানার শ্রমিকের,এ ঘখটনায় বিক৹ষোভ রেছেন ইও কার৾ানার ্শরমিকেরা ।,"[E, GHTNAAY, BIKSHBH, KRECHEN, OI, KAARKHAANAA...",E GHTNAAY BIKSHBH KRECHEN OI KAARKHAANAAR SHRM...
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[সফিপুর, মডার্ন, হাসপাতালের, জরুরি, বিভাগের, চ...","[(সফিপুর, NP), (মডার্ন, NC), (হাসপাতালের, NC),...",প্রথম পড়েছেন বলেন আল শ্রমিকের ডটকমক জরুর অসুস...,সৃিপুর মডজর্ন হাসপ৛তালের জুররি বি঻ভাগের চিহিত্...,"[SPHIPUR, MDAARN, HAASPAATAALER, JRURI, BIBHAA...",SPHIPUR MDAARN HAASPAATAALER JRURI BIBHAAGER C...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[এতে, আতঙ্কিত, হওয়ার, কিছু, নেই, ।]","[(এতে, PPR), (আতঙ্কিত, JJ), (হওয়ার, NC), (কিছ...",কিছ এত হওয়ার আতঙ্কিত নেই,এতঽ আতঙ্িকত হঝয়ার কছিু নে ।,"[ETE, AATNGKIT, HOYAAR, KICHU, NEI, ]",ETE AATNGKIT HOYAAR KICHU NEI
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,"[অসুস্থদের, চিকিত্সা, দেওয়া, হয়েছে, ।]","[(অসুস্থদের, NC), (চিকিত্সা, NC), (দেওয়া, NV)...",অসুস্থদের চিকিত্স দেওয,অসুস্থদর চিকতি্সা দেও়া হয়েছওে ।,"[ASUSTHDER, CIKITSAA, DEOYAA, HYECHE, ]",ASUSTHDER CIKITSAA DEOYAA HYECHE
...,...,...,...,...,...,...,...,...
603618,চীনের আনুহি প্রদেশে ঘটেছে এ ঘটনা।,চীনের আনুহি প্রদেশে ঘটেছে এ ঘটনা।,"[চীনের, আনুহি, প্রদেশে, ঘটেছে, এ, ঘটনা, ।]","[(চীনের, NP), (আনুহি, NC), (প্রদেশে, NC), (ঘটে...",চীনের প্রদেশ ঘটেছ এ ঘটন,৷চীনের পনুহি প্রদউশে ঘটে঵ে এ ঘটনাট ।,"[CIINER, AANUHI, PRDESHE, GHTECHE, E, GHTNAA, ]",CIINER AANUHI PRDESHE GHTECHE E GHTNAA
603619,রাষ্ট্রীয় বার্তা সংস্থা সিনহুয়া গতকাল বৃহস্পতি...,রাষ্ট্রীয় বার্তা সংস্থা সিনহুয়া গতকাল বৃহস্পতি...,"[রাষ্ট্রীয়, বার্তা, সংস্থা, সিনহুয়া, গতকাল, বৃ...","[(রাষ্ট্রীয়, JJ), (বার্তা, NC), (সংস্থা, NC), ...",রাষ্ট্রীয় বার্ত সংস্থ সিনহুয় গতকাল জানিয়েছ আনু...,রাষ্ট্ীয় বারত্া সংস্া িসনহুয়া ৫গতকাল বৃহস্তপিব...,"[RAASHTRII, BAARTAA, SNSTHAA, SINHUAA, GTKAAL,...",RAASHTRII BAARTAA SNSTHAA SINHUAA GTKAAL BRIHS...
603620,একপর্যায়ে চিকিৎসকেরা তার মৃত্যু হয়েছে বলে ঘোষণ...,একপর্যায়ে চিকিৎসকেরা তার মৃত্যু হয়েছে বলে ঘোষণ...,"[একপর্যায়ে, চিকিৎসকেরা, তার, মৃত্যু, হয়েছে, বল...","[(একপর্যায়ে, NC), (চিকিৎসকেরা, NC), (তার, PPR)...",একপর্যায় চিকিৎসকের মৃত্য হয়েছ বল ঘোষণ করেন,৻কপর্যায়ে চিিকৎসকেরা তরা মৃতআযু হয়েে বথে ৞ঘোষণ...,"[EKPRYAAE, CIKISKERAA, TAAR, MRITYU, HECHE, BL...",EKPRYAAE CIKISKERAA TAAR MRITYU HECHE BLE GHSH...
603621,প্রাদেশিক রাজধানী হেফেইতে অন্ত্যেষ্টিক্রিয়ার স...,প্রাদেশিক রাজধানী হেফেইতে অন্ত্যেষ্টিক্রিয়ার স...,"[প্রাদেশিক, রাজধানী, হেফেইতে, অন্ত্যেষ্টিক্রিয়...","[(প্রাদেশিক, JJ), (রাজধানী, NC), (হেফেইতে, VM)...",সঙ্গ আওয়াজ প্রাদেশিক শিশুটির বুধবার গত যুক্ত অ...,প্রােদশিক রাজধানীন হেফেতে অন্ত্যেষট্িক্রিয়ার ৫...,"[PRAADESHIK, RAAJDHAANII, HEPHEITE, ANTYESHTIK...",PRAADESHIK RAAJDHAANII HEPHEITE ANTYESHTIKRIAA...


Check the basic statistics of the corpus

In [21]:
# Check length of preprocessed text
df['cleaned_text'].apply(lambda x: len(x.split())).describe()

count    565398.000000
mean         12.969903
std          12.530220
min           1.000000
25%           8.000000
50%          11.000000
75%          16.000000
max        2336.000000
Name: cleaned_text, dtype: float64

In [22]:
# from nltk.tokenize import sent_tokenize

# # Number of tokens in each row
# df['num_tokens'] = df['tokens'].apply(len)

# # Number of sentences in each row
# df['num_sentences'] = df['sentence'].apply(lambda text: len(sent_tokenize(str(text))))

# # Total number of tokens in the dataset
# total_tokens = df['num_tokens'].sum()
# print("Total Tokens:", total_tokens)

# # Total number of sentences in the dataset
# total_sentences = df['num_sentences'].sum()
# print("Total Sentences:", total_sentences)

# # Summary of token count per row
# print("Token Count per Row Summary:")
# print(df['num_tokens'].describe())

# # Summary of sentence count per row
# print("Sentence Count per Row Summary:")
# print(df['num_sentences'].describe())